# Day 2 - Lab 1: AI-Generated System Design & Database Seeding

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Estimated Time:** 150 minutes

**Introduction:**
Welcome to Day 2! Today, we transition from *what* we're building to *how* we'll build it. In this lab, you will act as the lead architect for the Onboarding Tool. Your task is to use the PRD to define the data structure of the application and create a tangible database artifact that will be used for the rest of the course.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

In [47]:
import os
print(os.getcwd())

c:\Users\labadmin\Desktop\AG-AISOFTDEV\Labs\Day_02_Design_and_Architecture


## Step 1: Setup

We will load the `day1_prd.md` artifact from Day 1. This document is the primary source of truth for our project and provides the necessary context for the LLM to make intelligent design suggestions.

**Model Selection:**
Feel free to experiment with different models by changing the `model_name` in `setup_llm_client()`. Models with strong reasoning capabilities, like `gpt-4o`, `o3`, or `gemini-2.5-pro`, are excellent choices for design tasks.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the PRD from the `artifacts` directory.
- `save_artifact()`: To save the generated SQL schema and seed data.
- `clean_llm_output()`: To remove markdown fences from the generated SQL code.

In [48]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the PRD from Day 1
prd_content = load_artifact("artifacts/day1_prd.md")
if not prd_content:
    print("Warning: Could not load day1_prd.md. Lab may not function correctly.")

2025-09-23 12:00:01,411 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating the SQL Schema

**Task:** Use the PRD to generate a normalized SQL schema for the application.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a Database Administrator (DBA).
2.  Provide the `prd_content` as context.
3.  Ask the LLM to design a normalized SQL schema with at least two tables (e.g., `users` and `onboarding_tasks`).
4.  The output should be the raw `CREATE TABLE` statements.
5.  Save the generated SQL to `artifacts/schema.sql`.

In [49]:
# Write a prompt to generate the SQL schema from the PRD.
# Ensure prd_content is loaded before using it in the prompt.
if 'prd_content' not in globals():
    try:
        from utils import load_artifact
        prd_content = load_artifact("../../artifacts/day1_prd.md")
    except Exception as e:
        prd_content = ""
        print(f"Error loading PRD: {e}")

schema_prompt = f"""
You are an expert Database Administrator (DBA).

Your task is to design a normalized SQL schema for a new hire onboarding tool, based on the following Product Requirements Document (PRD):

{prd_content}

Requirements:
- Design a normalized SQL schema for the application.
- Include at least two tables, such as 'users' and 'onboarding_tasks'.
- Use appropriate data types and primary/foreign keys to establish relationships.
- Output only the raw SQL CREATE TABLE statements, with no explanations or markdown formatting.
- Use SQLite syntax: INTEGER PRIMARY KEY AUTOINCREMENT, TEXT, REAL, BLOB, and TIMESTAMP DEFAULT CURRENT_TIMESTAMP. Avoid SERIAL, TIMESTAMPTZ, ENUM, and ON DELETE/UPDATE SET NULL.
- Add IF NOT EXISTS to all CREATE TABLE statements.
"""

print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)

    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')

    # Optionally, further clean up for SQLite compatibility
    def sqlite_cleanup(sql):
        import re
        # Remove duplicate CREATE TABLEs
        tables = {}
        cleaned_lines = []
        for line in sql.splitlines():
            m = re.match(r'CREATE TABLE IF NOT EXISTS ([a-zA-Z0-9_]+)', line.strip())
            if m:
                table = m.group(1)
                if table in tables:
                    continue
                tables[table] = True
            cleaned_lines.append(line)
        sql = '\n'.join(cleaned_lines)
        # Replace SERIAL with INTEGER PRIMARY KEY AUTOINCREMENT
        sql = re.sub(r'SERIAL PRIMARY KEY', 'INTEGER PRIMARY KEY AUTOINCREMENT', sql)
        # Replace TIMESTAMPTZ with TIMESTAMP
        sql = re.sub(r'TIMESTAMPTZ', 'TIMESTAMP', sql)
        # Replace INT with INTEGER
        sql = re.sub(r'\bINT\b', 'INTEGER', sql)
        # Replace DECIMAL with REAL
        sql = re.sub(r'DECIMAL\([^)]+\)', 'REAL', sql)
        # Replace ENUM with TEXT
        sql = re.sub(r'ENUM\([^)]+\)', 'TEXT', sql)
        # Remove ON DELETE/UPDATE SET NULL
        sql = re.sub(r'ON DELETE SET NULL', '', sql)
        sql = re.sub(r'ON UPDATE SET NULL', '', sql)
        # Remove duplicate whitespace
        sql = re.sub(r'\n{2,}', '\n', sql)
        return sql

    cleaned_schema_sqlite = sqlite_cleanup(cleaned_schema)
    print(cleaned_schema_sqlite)

    # Save the cleaned schema, allowing overwrite
    save_artifact(cleaned_schema_sqlite, 'artifacts/schema.sql', overwrite=True)
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

--- Generating SQL Schema ---
CREATE TABLE IF NOT EXISTS roles (
    role_id INTEGER PRIMARY KEY AUTOINCREMENT,
    role_name TEXT NOT NULL UNIQUE
);
CREATE TABLE IF NOT EXISTS teams (
    team_id INTEGER PRIMARY KEY AUTOINCREMENT,
    team_name TEXT NOT NULL,
    description TEXT
);
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    password_hash TEXT NOT NULL,
    role_id INTEGER NOT NULL,
    manager_id INTEGER,
    team_id INTEGER,
    start_date TIMESTAMP,
    gamification_points INTEGER DEFAULT 0,
    profile_picture_url TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (role_id) REFERENCES roles (role_id),
    FOREIGN KEY (manager_id) REFERENCES users (user_id),
    FOREIGN KEY (team_id) REFERENCES teams (team_id)
);
CREATE TABLE IF NOT EXISTS onboarding_plans (
    plan_id INTEGER PRIMARY KEY AUTOINCREMENT,
    plan_name TEXT NOT 

### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Task:** Prompt the LLM to generate realistic seed data that conforms to the schema you just created.

**Instructions:**
1.  Create a new prompt.
2.  Provide both the `prd_content` and the `cleaned_schema` as context.
3.  Instruct the LLM to generate 5-10 realistic `INSERT` statements for your tables.
4.  The data should be relevant to a new hire onboarding tool (e.g., sample user names and task titles like "Complete HR Paperwork").
5.  Save the generated `INSERT` statements to `artifacts/seed_data.sql`.

In [51]:
# Write a prompt to generate realistic seed data.
seed_data_prompt = f"""
You are an expert Database Administrator (DBA).

Your task is to generate realistic SQL seed data for a new hire onboarding tool, based on the following context:

Product Requirements Document (PRD):
{prd_content}

SQL Schema:
{cleaned_schema}

Instructions:
- Generate 5-10 realistic SQL INSERT statements for each table in the schema.
- The data should be relevant to a new hire onboarding tool (e.g., sample user names, onboarding task titles like 'Complete HR Paperwork').
- Output only the raw SQL INSERT statements, with no explanations or markdown formatting.
- Use SQLite-compatible INSERT syntax only.
"""

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)

    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')

    # Optionally, further clean up for SQLite compatibility
    def sqlite_seed_cleanup(sql):
        import re
        # Remove SERIAL, TIMESTAMPTZ, ENUM, etc. from INSERTs if present
        sql = re.sub(r'SERIAL', '', sql)
        sql = re.sub(r'TIMESTAMPTZ', '', sql)
        sql = re.sub(r'ENUM\([^)]+\)', '', sql)
        # Remove duplicate whitespace
        sql = re.sub(r'\n{2,}', '\n', sql)
        return sql

    cleaned_seed_sqlite = sqlite_seed_cleanup(cleaned_seed_data)
    print(cleaned_seed_sqlite)

    # Save the cleaned seed data, allowing overwrite
    save_artifact(cleaned_seed_sqlite, 'artifacts/seed_data.sql', overwrite=True)
else:
    print("Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---
INSERT INTO roles (role_id, role_name) VALUES (1, 'HR Manager'), (2, 'Team Leader'), (3, 'New Hire'), (4, 'Employee');
INSERT INTO teams (team_id, team_name, description) VALUES (1, 'Engineering', 'Develops and maintains the company''s software products.'), (2, 'Marketing', 'Manages branding, advertising, and lead generation.'), (3, 'Sales', 'Drives revenue through direct sales and partnerships.'), (4, 'Human Resources', 'Manages employee lifecycle, benefits, and company culture.'), (5, 'Product Management', 'Defines product vision, strategy, and roadmap.');
INSERT INTO users (user_id, first_name, last_name, email, password_hash, role_id, manager_id, team_id, start_date, gamification_points, profile_picture_url) VALUES (1, 'Sarah', 'Chen', 'sarah.chen@example.com', 'hashed_password_1', 1, NULL, 4, '2022-01-15 09:00:00', 50, 'https://example.com/profiles/sarah.jpg'), (2, 'John', 'Miller', 'john.miller@example.com', 'hashed_password_2', 2, NULL, 1, '2022-03-2

### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Task:** This is a critical technical step. You will write a Python script to execute the generated SQL files, creating a live `onboarding.db` file that your application will use.

**Instructions:**
1.  Complete the `create_database` function below.
2.  The function should first connect to (and thus create) a SQLite database file named `artifacts/onboarding.db`.
3.  It should then open and execute the `schema.sql` file to create the tables.
4.  Finally, it should open and execute the `seed_data.sql` file to populate the tables.
5.  Use a `try...finally` block to ensure the database connection is always closed, even if an error occurs.

> **Hint:** The `try...finally` block is a crucial Python pattern. The code in the `finally` block will run whether the `try` block succeeds or fails, making it the perfect place to ensure resources like database connections are always closed.

**Expected Quality:** A physical `onboarding.db` file in your `artifacts` folder. This is a tangible asset that proves your design is valid and provides a concrete foundation for backend development.

In [52]:
def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    # TODO: Check if the schema file exists.
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return
    
    conn = None
    try:
        # TODO: Connect to the SQLite database. This will create the file if it doesn't exist.
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")
        
        # TODO: Read the content of the schema file using load_artifact.
        from utils import load_artifact
        schema_sql = load_artifact(schema_path)
        
        # TODO: Execute the schema SQL script.
        # Hint: Use cursor.executescript() for multi-statement SQL strings.
        cursor.executescript(schema_sql)
        print("Tables created successfully.")
        
        # TODO: Check if the seed data file exists. If it does, load and execute it.
        if os.path.exists(seed_path):
            # Load and execute the seed data
            seed_sql = load_artifact(seed_path)
            cursor.executescript(seed_sql)
            print("Seed data inserted successfully.")
        
        # TODO: Commit the changes to the database.
        conn.commit()
        print("Database changes committed.")
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        # TODO: Ensure the connection is closed if it was opened.
        if conn:
            conn.close()
            print("Database connection closed.")

# Define file paths
db_file = os.path.join(project_root, "artifacts", "onboarding.db")
schema_file = os.path.join(project_root, "artifacts", "schema.sql")
seed_file = os.path.join(project_root, "artifacts", "seed_data.sql")

# Execute the function
create_database(db_file, schema_file, seed_file)

Successfully connected to database at c:\Users\labadmin\Desktop\AG-AISOFTDEV\artifacts\onboarding.db
Tables created successfully.
Seed data inserted successfully.
Database changes committed.
Database connection closed.
Seed data inserted successfully.
Database changes committed.
Database connection closed.


## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.